In [1]:
import yaml
import sys
import traceback
import logging
import contextlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn

%matplotlib inline

from tqdm.notebook import tqdm
from typing import *

%load_ext autoreload
%autoreload 2

import os
current_dir = os.getcwd()
os.chdir("../src")
from dqnroute import *
os.chdir(current_dir)

logger = logging.getLogger(DQNROUTE_LOGGER)
TORCH_MODELS_DIR = '../torch_models'
LOG_DATA_DIR = '../logs/runs'
np.set_printoptions(linewidth=500)

_legend_txt_replace = {
    'networks': {
    'link_state': 'Shortest paths',
    'simple_q': 'Q-routing',
    'pred_q': 'PQ-routing',
    'glob_dyn': 'Global-dynamic',
    'dqn': 'DQN',
    'dqn_oneout': 'DQN (1-out)',
    'dqn_emb': 'DQN-LE',
    'centralized_simple': 'Centralized control'
    },
    'conveyors': {
    'link_state': 'Vyatkin-Black',
    'simple_q': 'Q-routing',
    'pred_q': 'PQ-routing',
    'glob_dyn': 'Global-dynamic',
    'dqn': 'DQN',
    'dqn_oneout': 'DQN (1-out)',
    'dqn_emb': 'DQN-LE',
    'centralized_simple': 'BSR'
    }
}

_targets = {'time': 'avg','energy': 'sum', 'collisions': 'sum'}

_ylabels = {
    'time': 'Mean delivery time',
    'energy': 'Total energy consumption',
    'collisions': 'Cargo collisions'
}

def print_sums(df):
    types = set(df['router_type'])
    for tp in types:
        x = df.loc[df['router_type']==tp, 'count'].sum()
        txt = _legend_txt_replace.get(tp, tp)
        print('  {}: {}'.format(txt, x))

def plot_data(data, meaning='time', figsize=(15,5), xlim=None, ylim=None,
              xlabel='Simulation time', ylabel=None,
              font_size=14, title=None, save_path=None,
              draw_collisions=False, context='networks', **kwargs):
    if 'time' not in data.columns:
        datas = split_dataframe(data, preserved_cols=['router_type', 'seed'])
        for tag, df in datas:
            if tag == 'collisions' and not draw_collisions:
                print('Number of collisions:')
                print_sums(df)
                continue
                
            xlim = kwargs.get(tag+'_xlim', xlim)
            ylim = kwargs.get(tag+'_ylim', ylim)
            save_path = kwargs.get(tag+'_save_path', save_path)
            plot_data(df, meaning=tag, figsize=figsize, xlim=xlim, ylim=ylim,
                      xlabel=xlabel, ylabel=ylabel, font_size=font_size,
                      title=title, save_path=save_path, context='conveyors')
        return 
    
    target = _targets[meaning]
    if ylabel is None:
        ylabel = _ylabels[meaning]
        
    fig = plt.figure(figsize=figsize)
    ax = sns.lineplot(x='time', y=target, hue='router_type', data=data,
                      err_kws={'alpha': 0.1})
    
    handles, labels = ax.get_legend_handles_labels()
    new_labels = list(map(lambda l: _legend_txt_replace[context].get(l, l), labels[1:]))
    ax.legend(handles=handles[1:], labels=new_labels, fontsize=font_size)
    
    ax.tick_params(axis='both', which='both', labelsize=int(font_size*0.75))
        
    if xlim is not None:
        ax.set_xlim(xlim)
    if ylim is not None:
        ax.set_ylim(ylim)
    if title is not None:
        ax.set_title(title)
    
    ax.set_xlabel(xlabel, fontsize=font_size)
    ax.set_ylabel(ylabel, fontsize=font_size)
    
    plt.show(fig)
    
    if save_path is not None:
        fig.savefig('../img/' + save_path, bbox_inches='tight')

def split_data(dct):
    results = []
    
    def add_res(i, key, val):
        while len(results) <= i:
            results.append({})
        results[i][key] = val
    
    for (key, vals) in dct.items():
        for (i, val) in enumerate(vals):
            add_res(i, key, val)
    return tuple(results)
    
def combine_launch_data(launch_data):
    dfs = []
    for (job_id, data) in launch_data.items():
        router_type, seed = un_job_id(job_id)
        df = data.copy()
        add_cols(df, router_type=router_type, seed=seed)
        dfs.append(df)
    return pd.concat(dfs, axis=0)

class DummyTqdmFile(object):
    """Dummy file-like that will write to tqdm"""
    file = None
    def __init__(self, file):
        self.file = file

    def write(self, x):
        # Avoid print() second call (useless \n)
        if len(x.rstrip()) > 0:
            tqdm.write(x, file=self.file)

    def flush(self):
        return getattr(self.file, "flush", lambda: None)()

@contextlib.contextmanager
def std_out_err_redirect_tqdm():
    orig_out_err = sys.stdout, sys.stderr
    try:
        sys.stdout, sys.stderr = map(DummyTqdmFile, orig_out_err)
        yield orig_out_err[0]
    # Relay exceptions
    except Exception as exc:
        raise exc
    # Always restore sys.stdout/err if necessary
    finally:
        sys.stdout, sys.stderr = orig_out_err

In [3]:
def run_single(file: str, router_type: str, random_seed: int, **kwargs):
    job_id = mk_job_id(router_type, random_seed)
    with tqdm(desc=job_id) as bar:
        queue = DummyProgressbarQueue(bar)
        runner = ConveyorsRunner(run_params=file, router_type=router_type,
                                 random_seed=random_seed, progress_queue=queue, **kwargs)
        event_series = runner.run(**kwargs)
    return event_series, runner

event_series, runner = run_single(file='../launches/conveyor_energy_test.yaml',
                                  router_type='dqn_emb', progress_step=500,
                                  ignore_saved=[True], random_seed=44)

In [4]:
#print(type(runner).mro()[:-1])
print(type(runner.world).mro()[:-1])
#print(type(runner.world.factory).mro()[:-1])
#print(type(runner.world.factory.sub_factory).mro()[:-1])
#print(type(runner.world.factory.sub_factory.topology_graph).mro()[:-1])
#print(runner.world.factory.sub_factory.training_mode)
#print(runner.world.factory.sub_factory.router_cfg)
#print(runner.world.factory.sub_factory.RouterClass)

#print(runner.world.layout)

#def node_key_to_routers(node_key):
#    return routers[node_key].routers

graph = runner.world.topology_graph
#print(dir(graph))
routers = runner.world.handlers
q_network = None
for node_key, router_keeper in routers.items():
    print("node", node_key, type(router_keeper).__name__)
    out_edges = graph.out_edges(node_key)
    out_nodes = [e[1] for e in out_edges]
    for router_key, router in router_keeper.routers.items():
        print("    router", router_key, type(router).__name__)
        #print(value.brain.ff_net)
        #print(router.nodes)
        q_network = router.brain
        node_repr = router._nodeRepr
        #print("    directed edges:", out_edges)
        print("    directed neighbor nodes:", out_nodes)
        neighbor_routers = [list(routers[node].routers.keys()) if node in routers.keys() else []
                            for node in out_nodes]
        neighbor_routers = sum(neighbor_routers, [])
        print("    directed neighbor routers:", neighbor_routers)

ff_network = q_network.ff_net

print(node_repr)
print(runner.world.topology_graph.edges)

[<class 'dqnroute.simulation.conveyors.ConveyorsEnvironment'>, <class 'dqnroute.simulation.common.MultiAgentEnv'>, <class 'dqnroute.utils.HasLog'>, <class 'dqnroute.utils.HasTime'>]
node ('sink', 0) RouterSink
    router ('router', 14) DQNRouterEmbConveyor
    directed neighbor nodes: []
    directed neighbor routers: []
node ('sink', 1) RouterSink
    router ('router', 15) DQNRouterEmbConveyor
    directed neighbor nodes: []
    directed neighbor routers: []
node ('sink', 2) RouterSink
    router ('router', 16) DQNRouterEmbConveyor
    directed neighbor nodes: []
    directed neighbor routers: []
node ('sink', 3) RouterSink
    router ('router', 17) DQNRouterEmbConveyor
    directed neighbor nodes: []
    directed neighbor routers: []
node ('source', 0) RouterSource
    router ('router', 18) DQNRouterEmbConveyor
    directed neighbor nodes: [('diverter', 0)]
    directed neighbor routers: [('router', 0)]
node ('source', 1) RouterSource
    router ('router', 19) DQNRouterEmbConveyor
  

In [13]:
# testing delivery
sources = [node_key for node_key, _ in routers.items() if node_key[0] == "source"]
sinks = [node_key for node_key, _ in routers.items() if node_key[0] == "sink"]
print(sources, sinks)

def get_out_nodes(node_key):
    return [e[1] for e in graph.out_edges(node_key)]

def get_final_router(node_key):
    """Returns dict{where_to_go: router_id}"""
    # if "source", "conveyor", "junction": return find_neighbors of the only child
    # if "sink", "diverter": it contains a router to return
    if node_key[0] in ["source", "conveyor", "junction"]:
        out_nodes = get_out_nodes(node_key)
        assert len(out_nodes) == 1, out_nodes
        return get_final_router(out_nodes[0])
    elif node_key[0] in ["sink", "diverter"]:
        r = list(routers[node_key].routers.keys())
        assert len(r) == 1
        return r[0]
    else:
        raise AssertionError()

[('source', 0), ('source', 1)] [('sink', 0), ('sink', 1), ('sink', 2), ('sink', 3)]


In [54]:
# find the reachability matrix

node_keys = sorted(list(runner.world.topology_graph.nodes))

# 1. initialize with self-reachability
reachable = {(k1, k2): k1 == k2 for k1 in node_keys for k2 in node_keys}
# 2. add transitions
for from_node in node_keys:
    #print(get_out_nodes(from_node))
    for to_node in get_out_nodes(from_node):
        reachable[from_node, to_node] = True
# 3. close with Floyd-Warshall
for i in node_keys:
    for j in node_keys:
        for k in node_keys:
            reachable[i, j] |= reachable[i, k] and reachable[k, j]

for from_node in node_keys:
    for to_node in node_keys:
        print(1 if reachable[from_node, to_node] else 0, end="")
    print(f" # from {from_node}")

11000000011011100000 # from ('diverter', 0)
01000001110000111100 # from ('diverter', 1)
00110000011011000000 # from ('diverter', 2)
01010001110010111100 # from ('diverter', 3)
00001100000100001100 # from ('diverter', 4)
00000110000100111000 # from ('diverter', 5)
00000010000000110000 # from ('diverter', 6)
00000001100000011100 # from ('diverter', 7)
00000000100000001100 # from ('diverter', 8)
00000001110000011100 # from ('junction', 0)
00001110001100111100 # from ('junction', 1)
00000010000100110000 # from ('junction', 2)
01000001110010111100 # from ('junction', 3)
00001110001101111100 # from ('junction', 4)
00000000000000100000 # from ('sink', 0)
00000000000000010000 # from ('sink', 1)
00000000000000001000 # from ('sink', 2)
00000000000000000100 # from ('sink', 3)
11000001111111111110 # from ('source', 0)
00110001111111111101 # from ('source', 1)


In [56]:
# test routing
for source in sources:
    for sink in sinks:
        print(f"Testing delivery from {source} to {sink}...")
        sink_router = get_final_router(sink)
        current_node = source
        visited_nodes = set()
        while True:
            if current_node in visited_nodes:
                print("    FAIL due to cycle")
                break
            visited_nodes.add(current_node)
            print("    in:", current_node)
            if current_node[0] == "sink":
                if current_node == sink:
                    print("    OK")
                else:
                    print("    FAIL due to wrong destination")
                break
            elif current_node[0] in ["source", "junction"]:
                out_nodes = get_out_nodes(current_node)
                assert len(out_nodes) == 1
                current_node = out_nodes[0]
            elif current_node[0] == "diverter":
                out_nodes = get_out_nodes(current_node)
                # leave only nodes from which the sink is reachable
                out_nodes = [out_node for out_node in out_nodes if reachable[out_node, sink]]
                print("        current candidate neighbors:", out_nodes)
                current_router = list(routers[current_node].routers.keys())
                assert len(current_router) == 1
                current_router = current_router[0]
                #assert isinstance(current_router, agents.conveyors.diverter.RouterDiverter)
                out_routers = [get_final_router(node) for node in out_nodes]
                print("        routers of these neighbors:", out_routers)
                # get the distribution
                dst = torch.FloatTensor([node_repr(sink_router[1])])
                addr = torch.FloatTensor([node_repr(current_router[1])])
                q_values = []
                for out_router in out_routers:
                    neighbor = torch.FloatTensor([node_repr(out_router[1])])
                    with torch.no_grad():
                        q = q_network.forward(addr, dst, neighbor).item()
                        print(f"        Q({current_router} -> {out_router} | {sink_router}) = {q:.4f}")
                        q_values += [q]
                best_neighbor_index = np.argmax(np.array(q_values))
                current_node = out_nodes[best_neighbor_index]
                #raise AssertionError()
            else:
                raise AssertionError()

Testing delivery from ('source', 0) to ('sink', 0)...
    in: ('source', 0)
    in: ('diverter', 0)
        current candidate neighbors: [('junction', 3), ('junction', 4)]
        routers of these neighbors: [('router', 1), ('router', 4)]
        Q(('router', 0) -> ('router', 1) | ('router', 14)) = -14.1944
        Q(('router', 0) -> ('router', 4) | ('router', 14)) = -16.6087
    in: ('junction', 3)
    in: ('diverter', 1)
        current candidate neighbors: [('sink', 0)]
        routers of these neighbors: [('router', 14)]
        Q(('router', 1) -> ('router', 14) | ('router', 14)) = -6.5029
    in: ('sink', 0)
    OK
Testing delivery from ('source', 0) to ('sink', 1)...
    in: ('source', 0)
    in: ('diverter', 0)
        current candidate neighbors: [('junction', 3), ('junction', 4)]
        routers of these neighbors: [('router', 1), ('router', 4)]
        Q(('router', 0) -> ('router', 1) | ('router', 15)) = -17.2787
        Q(('router', 0) -> ('router', 4) | ('router', 15)) = -2

In [171]:
num_nodes = 3#20
with torch.no_grad():
    for dst_index in range(num_nodes):
        dst = torch.FloatTensor([node_repr(dst_index)])
        for addr_index in range(num_nodes):
            addr = torch.FloatTensor([node_repr(addr_index)])
            for neighbor_index in range(num_nodes):
                neighbour = torch.FloatTensor([node_repr(neighbor_index)])
                print(f"{addr_index} directing to {neighbor_index} with target {dst_index}:",
                      q_network.forward(addr, dst, neighbour))

0 directing to 0 with target 0: tensor([[1.3856]])
0 directing to 1 with target 0: tensor([[-18.9589]])
0 directing to 2 with target 0: tensor([[-12.7015]])
1 directing to 0 with target 0: tensor([[-13.3700]])
1 directing to 1 with target 0: tensor([[-14.8461]])
1 directing to 2 with target 0: tensor([[-17.3598]])
2 directing to 0 with target 0: tensor([[-24.1153]])
2 directing to 1 with target 0: tensor([[-29.8411]])
2 directing to 2 with target 0: tensor([[-22.9305]])
0 directing to 0 with target 1: tensor([[-22.9985]])
0 directing to 1 with target 1: tensor([[-18.4110]])
0 directing to 2 with target 1: tensor([[-18.2000]])
1 directing to 0 with target 1: tensor([[-19.9836]])
1 directing to 1 with target 1: tensor([[1.3856]])
1 directing to 2 with target 1: tensor([[-15.5730]])
2 directing to 0 with target 1: tensor([[-32.5020]])
2 directing to 1 with target 1: tensor([[-19.0971]])
2 directing to 2 with target 1: tensor([[-18.8911]])
0 directing to 0 with target 2: tensor([[-32.8053]